# Churn Prediction with Machine Learning
## Thiago Pádua de Carvalho - 2020007066

### Introdução - O que é Churn?
Churn é o termo que define a taxa de evasão de clientes de um serviço ou produto fornecido por uma empresa. Ele é um elemento fundamental para medir a fidelidade e a satisfação dos consumidores, um indicador importante para a saúde financeira de uma companhia, análise de situação no mercado e consequente tomada de decisões estratégicas.

Entender o comportamento dos compradores e prever sua evasão é um desafio para as empresas, pois esse fenômeno está diretamente relacionado com a perda de receita e viabilidade do negócio. Sendo assim é fundamental ter informações sobre o que leva um cliente a cancelar um serviço para identificar pontos de falha, tal como investir assertivamente em ações de retenção.

Com isso em mente, o objetivo deste trabalho é construir um modelo de Machine Learning para prever o churn, tal como identificar aqueles atributos que mais contribuem para a evasão de clientes, permitindo uma visão ampla e detalhada sobre o problema para a posterior tomada de decisões.

### Importação das bibliotecas

In [33]:
import pandas as pd
import matplotlib.pyplot as plt

## Conhecendo os dados
O dataset utilizado neste trabalho é composto por dois arquivos CSV: um para treinamento e outro para teste.

O arquivo de treinamento para o conjunto de dados de churn contém uma coleção de 440.882 registros de clientes, juntamente com suas respectivas características e rótulos binários. Cada linha no arquivo de treinamento representa um cliente e inclui as seguintes características: idade, gênero, tempo de serviço, frequência de uso, chamadas de suporte, atraso no pagamento, tipo de assinatura, duração do contrato, gasto total e última interação.

O arquivo de teste para o conjunto de dados de churn consiste em 64.374 registros de clientes e serve como um conjunto de dados separado para avaliar o desempenho e a capacidade de generalização dos modelos de previsão de rotatividade treinados. Cada linha no arquivo de teste corresponde a um cliente e contém o mesmo conjunto de características que o arquivo de treinamento. No entanto, os rótulos de churn não estão incluídos no arquivo de teste, pois são usados para avaliar a precisão e a eficácia dos modelos. O arquivo de teste permite avaliar o poder preditivo dos modelos treinados em dados não vistos e obter insights sobre o quão bem eles se generalizam para novos clientes.

Fonte dos dados: https://www.kaggle.com/datasets/muhammadshahidazeem/customer-churn-dataset?resource=download&select=customer_churn_dataset-testing-master.csv

### Feature Columns
Atributos numéricos:
- **Age**: Idade
- **Tenure**: Duração em Meses que o cliente consome o serviço
- **Usage Frequency**: número de vezes que o cliente usou o serviço no mês anterior
- **Support Calls**: número de chamadas de suporte feitas pelo cliente no mês anterior
- **Payment Delay**: número de dias de atraso no pagamento do cliente para o mês anteriror
- **Total Spend**: gasto total do cliente com o serviço
- **Last Interaction**: número de dias desde a última interação do cliente com o serviço

Atributos categóricos
- **Gender**: Sexo do cliente (M/F)
- **Subscription Type**: Tipo de assinatura do cliente (Standard, Basic, Premium)
- **Contract Length**: Tipo de duração do contrato do cliente (Annual, Monthly, Quarterly)



### Carregamento dos dados

In [34]:
trainingData = pd.read_csv('data/customer_churn_dataset-training-master.csv')

In [35]:
testData = pd.read_csv('data/customer_churn_dataset-testing-master.csv')

### Pré-processamento

In [36]:
trainingData = trainingData.dropna()
trainingData.drop(['CustomerID'], axis=1, inplace=True)
X_train, y_train = trainingData.drop('Churn', axis=1), trainingData['Churn']

In [37]:
testData = testData.dropna()
testData.drop(['CustomerID'], axis=1, inplace=True)
X_test, y_test = testData.drop('Churn', axis=1), testData['Churn']

In [38]:
trainingData.describe()

,Age,Tenure,Usage Frequency,Support Calls,Payment Delay,Total Spend,Last Interaction,Churn
count,440832.000000,440832.000000,440832.000000,440832.000000,440832.000000,440832.000000,440832.000000,440832.000000
mean,39.373153,31.256336,15.807494,3.604437,12.965722,631.616223,14.480868,0.567107
std,12.442369,17.255727,8.586242,3.070218,8.258063,240.803001,8.596208,0.495477
min,18.000000,1.000000,1.000000,0.000000,0.000000,100.000000,1.000000,0.000000
25%,29.000000,16.000000,9.000000,1.000000,6.000000,480.000000,7.000000,0.000000
50%,39.000000,32.000000,16.000000,3.000000,12.000000,661.000000,14.000000,1.000000
75%,48.000000,46.000000,23.000000,6.000000,19.000000,830.000000,22.000000,1.000000
max,65.000000,60.000000,30.000000,10.000000,30.000000,1000.000000,30.000000,1.000000


### Análise Exploratória

array(['Annual', 'Monthly', 'Quarterly'], dtype=object)